#Классификация

В прошлый раз мы рассмотрели предсказание вероятностей для 2 классов.

Что делать в случае нескольких классов?

###Использовать несколько ,бинарных классификаторов

    Для каждого класса обучим классификатор, который делит объекты на заданный класс и все остальные. Напрмер для первого классификатора разделим вск объекты на 2 класса: первый и все остальные. Далее обучим его как в прошлый раз. Аналогично обучим остальные классификаторы для других классов.
    Для предсказания класса нового объекта будем искать максимум из предсказанных вероятностей среди всех классификаторов. 
    
    
###Cпециальную функцию, которя называется softmax.

Пусть  у нас 3 класса, нам надо предсказать вероятности для каждого из них.  Вероятности должны лежатьв диапазоне [0, 1] и их сумма должна быть равна 1.

Введем набор весов для каждого класса: для класса 1 введем ветор $\vec{\omega_1}$, для класса 2 - $\vec{\omega_2}$ и $\vec{\omega_3}$ для класса 3. Эти 3 вектора можно обединить в матрицу параметров:
$$W=(\vec{\omega_1},\;\vec{\omega_2},\;\vec{\omega_3})$$


Мы можем посчитать для каждого примера три выражения: $\vec{x}\cdot\vec{\omega_1}$, $\vec{x}\cdot\vec{\omega_2}$ и Тогда мы можем посчитать для каждого примера три выражения: $\vec{x}\cdot\vec{\omega_1}$,  $\vec{x}\cdot\vec{\omega_2}$ и $\vec{x}\cdot\vec{\omega_3}$.

В матричной форме:
$$margin = W\cdot\vec{x}$$
Получим вектор из 3 компонент.

Однако эти выражения могут принимать значения в диапазоне $(-\infty, \infty)$. Преобразуем их в верояности:
$$p_1=\frac{e^{\vec{x}\cdot\vec{\omega_1}}}{e^{\vec{x}\cdot\vec{\omega_1}}+e^{\vec{x}\cdot\vec{\omega_2}}+e^{\vec{x}\cdot\vec{\omega_3}}}$$
$$p_2=\frac{e^{\vec{x}\cdot\vec{\omega_2}}}{e^{\vec{x}\cdot\vec{\omega_1}}+e^{\vec{x}\cdot\vec{\omega_2}}+e^{\vec{x}\cdot\vec{\omega_3}}}$$
$$p_3=\frac{e^{\vec{x}\cdot\vec{\omega_3}}}{e^{\vec{x}\cdot\vec{\omega_1}}+e^{\vec{x}\cdot\vec{\omega_2}}+e^{\vec{x}\cdot\vec{\omega_3}}}$$

Можно убедиться, что такие выражения положительны и в сумме дают 1.
Эти три вероятности обединяют в вектор $\vec{p}$:
$$\vec{p}=(p_1,\;p_2,\;p_3)$$



Для обучения модели представим метки классов в виде векторов, для первого класса:
$$\vec{y_1}=(1,\;0,\;0)$$
$$\vec{y_2}=(0,\;1,\;0)$$
$$\vec{y_3}=(0,\;0,\;1)$$


В качестве функции потерь мы можем использовать стандартную среднюю разность квадратов. Для каждого примера мы знаем корректный класс, преобразуем его в соответствующий вектор и обозначи $\vec{y_i}$. Тогда можем посчитать отклонение каждой компоненты:
$$L=\frac{1}{N}\sum_i ((p_{i1}-y_{i1})^2+(p_{i2}-y_{i2})^2+(p_{i3}-y_{i3})^2)$$

На практике используют другую функцию потерь, которая дает лучшие результаты при обучении: «логлосс» (logloss / log_loss), перекрёстной / кросс-энтропией (Cross Entropy).

Для одного объекта:
$$L_i  = \sum_j -y_ij\cdot log(p_{ij})$$
В случае трех классов:
$$L_i  = -y_{i1}\cdot log(p_{i1})-y_{i2}\cdot log(p_{i2})-y_{i3}\cdot log(p_{i3})$$
Полная функция потерь:
$$L=\frac{1}{N}\sum_i L_i$$

p13 - вероятность того, что 1ый обект принадлежит классу 3
p52 - вероятность того, что 5ый обект принадлежит классу 2

p_i3 - вероятность того, что i-ый обект принадлежит классу 3

p_i2 - вероятность того, что i-ый обект принадлежит классу 2

p_i1 - вероятность того, что i-ый обект принадлежит классу 1

p_ij - вероятность того, что i-ый обект принадлежит классу j 


In [31]:
# посчитать кросс энтропию для заданных данных и весов
import numpy as np

n = 20

# генерируем данные
max_range = 10
X = (np.random.random( (n,4) )-0.5) * max_range 
y = np.random.randint(0,3,(n))

# генерируем данные
W = np.random.random((4,3))

scalar = np.dot(X, W)
print("Скалярные произведения: ", scalar.shape)

exps = np.exp(scalar)
print("Экспоненты от скалярных произведений: ", exps.shape)

# суммируем вдоль оси "1", то есть по столбцам 
sums_of_exp = np.sum(exps, axis=1)
print("Знаменатели: ", sums_of_exp.shape)

# вычисляем вероятности
# создаем массив зваполненный нулями
probabilities0 = np.zeros((n,3))
probabilities1 = np.zeros((n,3))
probabilities2 = np.zeros((n,3))

# не оптимальный вариант
# перебираем строчки, корторые содержат экспоненты
# for i in range(len(sums)):
#   перебераем все элементы в строке
#   for j in range(len(probabilities1[i])):
#        probabilities0[i][j] = exps[i][j] / sums_of_exp[i]

# Более оптимальный вариант:
# перебираем строчки, корторые содержат экспоненты
for i in range(len(sums_of_exp)):
    # делим i-ую строчку с экспонентами, на i-ую сумму и записываем результат в i-ую строку вероятностей
    probabilities1[i] = exps[i] / sums_of_exp[i]
    
# Самый оптимальный вариант с broadcating
probabilities2 = exps / sums_of_exp.reshape(-1, 1)

# проверки, можно раскомментировать
# суммируем вероятности того, что объект принадлежит какому-то из классов. Должны получить 1 для каждого объекта
# print(np.sum(probabilities1, axis=1))
# print(np.sum(probabilities2, axis=1))

# при совпадении должны быть все нули
# print(probabilities1 - probabilities2)


# создаем для каждого обекта вектор с тремя элементами, для начала заполняем единицу
y_vec0 = np.zeros((n,3))
y_vec1 = np.zeros((n,3))

# в каждом ряду проставляем 1 в нужном классе
for i in range(n):
    # если метка класса 0, значит ставим один в нулевую ячйку
    if y[i] == 0:
        y_vec0[i, 0] = 1
    # если метка класса 1, значит ставим один в первую ячйку
    if y[i] == 1:
        y_vec0[i, 1] = 1
    # если метка класса 2, значит ставим один в вторую ячйку
    if y[i] == 2:
        y_vec0[i, 2] = 1

# теперь y_vec0 содержит метки классов в виде векторов

# попробуйте:
# заполнить y_vec1 с помощь цикла, но без if'ов

# при совпадении должны быть все нули
# print(y_vec0 - y_vec1)
        
# logloss
L = None

Скалярные произведения:  (20, 3)
Экспоненты от скалярных произведений:  (20, 3)
Знаменатели:  (20,)
